In [1]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gpt-4o")

with open("../app/scripts/main.py", "r") as f:
    main_script = f.read()

with open("../app/db_schema.sql", "r") as f:
    db_schema = f.read()

2025-10-02 18:11:03,493 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=gpt-4o latency_ms=None artifacts_path=None


In [4]:
happy_path_tests_prompt = f'''
You are an expert Python developer and QA Engineer.
- Given the following FastAPI application code and SQL schema, generate comprehensive happy path tests using pytest and FastAPI's TestClient.
- Ensure the tests cover all endpoints and typical use cases.
- Do not include any negative tests or edge cases; focus solely on the expected successful operations.
- Your response should be only valid Python code that can be executed as a test suite.
- Ensure there is a if name == "main" block to run the tests when the script is executed directly.
- Use the following SQL schema for database context:

<db schema>
{db_schema}
</db schema>

<application code>
{main_script}
</application code>
'''

print("--- Generating Happy Path Tests ---")

generated_happy_path_tests = get_completion(happy_path_tests_prompt, client, model_name, api_provider)
cleaned_tests = clean_llm_output(generated_happy_path_tests, language='python')
print(cleaned_tests)
# (cleaned_tests, "tests/test_main_simple.py", overwrite=True)

with open("../app/scripts/simple_test.py", "w") as f:
    f.write(cleaned_tests)

--- Generating Happy Path Tests ---
import pytest
from fastapi.testclient import TestClient
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from app import app, get_db, Base

# Set up the test database
SQLALCHEMY_DATABASE_URL = "sqlite:///./test_recruitment_app.db"
engine = create_engine(
    SQLALCHEMY_DATABASE_URL, connect_args={"check_same_thread": False}
)
TestingSessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Create a new database session for testing
Base.metadata.create_all(bind=engine)

# Override the get_db dependency to use the test database
def override_get_db():
    try:
        db = TestingSessionLocal()
        yield db
    finally:
        db.close()

app.dependency_overrides[get_db] = override_get_db
client = TestClient(app)

@pytest.fixture(autouse=True)
def run_around_tests():
    # Before each test, clear the database
    Base.metadata.drop_all(bind=engine)
    Base.metadata.create_all(bind=engine)
    yield


In [10]:
edge_case_tests_prompt = f"""
You are an expert Python developer and QA Engineer.
- Given the following FastAPI application code, SQL schema, and existing test code, add comprehensive edge case tests to the existing test code using pytest and FastAPI's TestClient.
- Ensure the tests cover edge cases, error handling, and invalid inputs for all endpoints.
- Your response should be only valid Python code that can be executed as a test suite.
- Use the following SQL schema for database context:
{db_schema}
- Use the following FastAPI application code:
{main_script}
- Use the following existing test code as a reference:
{cleaned_tests}
"""

print("--- Generating Edge Case Tests ---")
if main_script:
    generated_edge_case_tests = get_completion(edge_case_tests_prompt, client, model_name, api_provider)
    cleaned_edge_case_tests = clean_llm_output(generated_edge_case_tests, language='python')
    print(cleaned_edge_case_tests)
    with open("../app/scripts/app_test/main_test_edge.py", "w") as f:
        f.write(cleaned_edge_case_tests)
else:
    print("Skipping test generation because app code is missing.")

--- Generating Edge Case Tests ---
import pytest
from fastapi.testclient import TestClient
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.pool import StaticPool
import sys
import os

# Add the current directory to the path to allow for imports
# This is necessary for the `if __name__ == "__main__"` block to work correctly
sys.path.insert(0, os.path.abspath(os.path.dirname(__file__)))

# --- FastAPI Application Code ---
# The provided application code is placed here directly to create a single, executable file.

from __future__ import annotations

import uvicorn
from datetime import datetime
from enum import Enum
from typing import List, Optional

from fastapi import FastAPI, HTTPException, status, Depends
from pydantic import BaseModel, Field, EmailStr, ConfigDict
from sqlalchemy import (
    create_engine as create_prod_engine,
    Column,
    ForeignKey,
    Integer,
    Table,
    Text,
    UniqueConstraint,
    text,
)
from sqlalchemy.e

In [11]:
db_fixture_prompt = f"""
You are an expert Python developer and QA Engineer.
- Given the following FastAPI application code, SQL schema, and existing test code, generate a pytest fixture that sets up and tears down an isolated duplicate test database for use in the tests.
- The fixture should create a new test database, apply the schema, and provide a test database session to the tests.
- The code must override the 'get_db' dependency in the FastAPI app to use the test database session.
- Refine the existing test code to utilize this fixture for database interactions.
- Your response should be only valid Python code that can be executed as a pytest fixture.
- Ensure the fixture properly cleans up the test database after tests are run.
- If you break the connection to the existing application, I will fire you, and you will never work in this industry again.
- Use the following SQL schema for database context:
{db_schema}
- Use the following FastAPI application code:
{main_script}
- Use the following existing test code as a reference:
{cleaned_edge_case_tests}
"""

print("--- Generating Pytest DB Fixture ---")
if main_script and cleaned_edge_case_tests:
    generated_db_fixture = get_completion(db_fixture_prompt, client, model_name, api_provider)
    cleaned_fixture = clean_llm_output(generated_db_fixture, language='python')
    print(cleaned_fixture)
    with open("../app/scripts/app_test/db_fixture.py", "w") as f:
        f.write(cleaned_fixture)
else:
    print("Skipping fixture generation because app context is missing.")

--- Generating Pytest DB Fixture ---
import pytest
from fastapi.testclient import TestClient
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import sys
import os

# Add the current directory to the path to allow for imports
# This is necessary for the `if __name__ == "__main__"` block to work correctly
sys.path.insert(0, os.path.abspath(os.path.dirname(__file__)))

# --- FastAPI Application Code ---
# The provided application code is placed here directly to create a single, executable file.

from __future__ import annotations

import uvicorn
from datetime import datetime
from enum import Enum
from typing import List, Optional

from fastapi import FastAPI, HTTPException, status, Depends
from pydantic import BaseModel, Field, EmailStr, ConfigDict
from sqlalchemy import (
    create_engine as create_prod_engine,
    Column,
    ForeignKey,
    Integer,
    Table,
    Text,
    UniqueConstraint,
    text,
)
from sqlalchemy.exc import IntegrityError
from sqlalch